### Imports ###
The Qiskit job watcher magic string is useful, and surprisingly poorly documented


In [ ]:
import time

import qiskitpool

from qiskit import IBMQ, QuantumCircuit
import qiskit.tools.jupyter

IBMQ.load_account()
provider = IBMQ.get_provider(group='open', project='main')

%qiskit_job_watcher

## Single circuit trial

In [ ]:
pool = qiskitpool.QPool(provider)
backend = provider.get_backend('simulator_statevector')

n_shots = 10

circ = QuantumCircuit(1, 1)
circ.x(0)
circ.measure(0, 0)

job = pool(circ, backend, shots=n_shots)

Print the pool to see what it's up to

In [ ]:
pool

In [ ]:
while not job.poll():
    time.sleep(2)
    

assert(job.result().get_counts()['1'] == n_shots)

## 20 Circuits trial
You can increase this, but the IBMQ backend may throttle your job submissions

In [ ]:
pool = qiskitpool.QPool(provider)
backend = provider.get_backend('simulator_statevector')

n_shots = 10
n_rounds = 20

circ = QuantumCircuit(1, 1)
circ.x(0)
circ.measure(0, 0)

# Create jobs
jobs = [pool(circ, backend, shots=n_shots) for i in range(n_rounds)]

### Print the pool while it is running to see what it's up to

In [ ]:
print(pool)

Wait for the jobs to finish and then check the results

In [ ]:
# Wait for jobs to finish
n_complete = 0
while n_complete < n_rounds - 1:
    n_complete = 0
    for job in jobs:
        if job.poll():
            n_complete += 1

for job in jobs:
    assert(job.poll())
    assert(job.result().get_counts()['1'] == n_shots)

Get the list of jobs run by that pool

In [ ]:
pool['simulator_statevector'].finished_jobs

Get a particular job by the order in which the jobs were queued.
This is useful in the event that you lose the reference to a job.

In [ ]:
pool['simulator_statevector'][0]

### Safely shut down the pool

In [ ]:
pool.join()